In [ ]:
!pip install mimikit

In [ ]:
from mimikit.freqnet import FreqNet
from mimikit.data import Database
from mimikit.utils import audio
from mimikit import get_trainer, NeptuneConnector

In [ ]:
# neptune :

nep_con = NeptuneConnector(user="k-tonal",
                           setup=dict(db="data-and-base-notebooks/DAT-27",
                                      model="experiment-1-B"))

db_name = "genoel-mix.h5"

# paths

path_to_db = "./" + db_name
path_to_model = "./model"


In [ ]:
# get your data

db = nep_con.download_database("db", db_name)
db.metadata

### Varying Parameters

In [ ]:
params = {
    # from quick to slow with accum left:
    0: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=0,
        accum_outputs=1,
    ),
    1: dict(
        with_skip_conv=False,
        with_residual_conv=True,
        pad_input=0,
        accum_outputs=1,
    ),
    2:  dict(
        with_skip_conv=True,
        with_residual_conv=False,
        pad_input=0,
        accum_outputs=1,
    ),
    3:  dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=0,
        accum_outputs=1,
    ),
    # quick with learned_padding left or right
    4: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=1,
        learn_padding=True,
    ),
    5: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=-1,
        learn_padding=True,
    ),
    # skips with learned_padding left or right
    6:  dict(
        with_skip_conv=True,
        with_residual_conv=False,
        pad_input=1,
        learn_padding=True,
    ),
    7:  dict(
        with_skip_conv=True,
        with_residual_conv=False,
        pad_input=-1,
        learn_padding=True,
    ),
    # full with learned_padding left or right
    8: dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=1,
        learn_padding=True,
    ),
    9: dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=-1,
        learn_padding=True,
    ),
}

In [ ]:
# pick a parametrization :

PARAMS_INDEX = 9

# if you want ilocs (files in the db) put them in this list :
ILOCS = []


# build a model and its trainer

model = FreqNet(
    data_object=db.fft.get(db.metadata.iloc[ILOCS]) if any(ILOCS) else db.fft,
    splits=[.85, .15],
     model_dim=1024,
     groups=1,
     n_layers=(4,),
    kernel_size=3,
    **params[PARAMS_INDEX]
     )

trainer = get_trainer(model=model,
                     max_epochs=100,
                     epochs=[99],
                     # try everything in 16-bits precision
                     precision=16,
                     root_dir=path_to_model,
                     neptune_connector=nep_con,
                     )

# log some infos to the neptune experiment :

nept_exp = [exp for exp in trainer.logger.experiment if exp.__class__.__name__ == "Experiment"]
if any(nept_exp):
    nept_exp[0].append_tags("INDEX=" + str(PARAMS_INDEX),
                         "precision=16",
                         "FreqNet",
                         "db_size=" + str(model.data.shape[0]),
                         *[k + "=" + str(v) for k, v in params[PARAMS_INDEX].items()])
    nept_exp[0].log_text("DB", db_name)
    nept_exp[0].log_text("INDEX", str(PARAMS_INDEX))
    nept_exp[0].log_text("db-name", db_name)
    nept_exp[0].log_text("db-id", nep_con.setup["db"])
    nept_exp[0].log_text("db-iloc", str(ILOCS) if any(ILOCS) else "")

# Train :

trainer.fit(model)

# upload checkpoints and logs

nep_con.upload_model("model", model, artifacts=("states", "logs"))

### try some prompts

In [ ]:
from random import randint

n_prompts = 2
prompts_indices = [randint(0, model.data.shape[0]-64) for _ in range(n_prompts)]

for n, i in enumerate(prompts_indices):
    output = model.generate(model.data[i:i+64],
                            n_steps=2048)
    
    print("prompt", n, "index", i)
    
    audio(output.squeeze().numpy())


### log a prompt you like

In [ ]:
i = 1234

model.log_audio("prompt=" + str(i), model.generate(model.data[i:i+64], n_steps=2048))

### Download an already trained model

In [ ]:
nep_con.setup["trained"] = "experiment-1-B/MOD-134"

nep_con.download_experiment("trained", destination=path_to_model, artifacts="states/")

### Reload a checkpoint

In [ ]:
# replace the "X" in the next line with the (saved) epoch you want to load

path_to_ckpt = path_to_model + "MOD-134/states/epoch=X.ckpt"

model = FreqNet.load_from_checkpoint(path_to_ckpt, data_object=db.fft)